In [1]:
import pandas as pd

# Try reading with 'latin1'
df = pd.read_csv("data_clean.csv",encoding="latin1")


In [2]:
df.columns = df.columns.str.replace("'", "").str.strip()

In [3]:
df.head()

,job_title_short,job_location,job_schedule_type,job_work_from_home,job_country,salary_year_avg,company_name,S.No,analyst_tools,libraries,cloud,databases,other,programming,webframeworks,os,async,sync
0,Data Scientist,"San José Province, San José, Costa Rica",Full-time,False,Costa Rica,109500.0,Netskope,1,['excel'],['gdpr'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Data Engineer,"Arlington, VA",Full-time,False,Sudan,140000.0,Intelletec,2,['tableau'],['spark'],['oracle'],"['mongodb', 'mysql', 'mariadb']",['kubernetes'],"['mongodb', 'python', 'r', 'sql']",NaN,NaN,NaN,NaN
2,Data Engineer,Anywhere,Full-time,True,United States,120000.0,Apex Systems,3,NaN,NaN,NaN,NaN,NaN,"['sql', 'python']",NaN,NaN,NaN,NaN
3,Data Scientist,"Mountain View, CA",Full-time,False,United States,228222.0,TikTok,4,NaN,NaN,NaN,NaN,NaN,"['sql', 'r', 'python']",['express'],NaN,NaN,NaN
4,Data Analyst,Anywhere,Full-time,True,United States,89000.0,Get It Recruit - Transportation,5,"['alteryx', 'tableau']",NaN,NaN,NaN,NaN,"['python', 'r']",NaN,NaN,NaN,NaN


In [4]:
df.columns

Index(['job_title_short', 'job_location', 'job_schedule_type',
       'job_work_from_home', 'job_country', 'salary_year_avg', 'company_name',
       'S.No', 'analyst_tools', 'libraries', 'cloud', 'databases', 'other',
       'programming', 'webframeworks', 'os', 'async', 'sync'],
      dtype='object')

In [5]:
print(df.dtypes)

job_title_short        object
job_location           object
job_schedule_type      object
job_work_from_home       bool
job_country            object
salary_year_avg       float64
company_name           object
S.No                    int64
analyst_tools          object
libraries              object
cloud                  object
databases              object
other                  object
programming            object
webframeworks          object
os                     object
async                  object
sync                   object
dtype: object


SKILL TO CATEGORY MAPPING

In [6]:
import pandas as pd
import ast
from collections import defaultdict

# Load dataset
df = pd.read_csv("data_clean.csv",encoding="latin1")
df.columns = df.columns.str.replace("'", "").str.strip()
# Ensure column names are clean (remove spaces)
df.columns = df.columns.str.strip()

# Check actual column names
print("✅ Actual Column Names:", df.columns.tolist())

# Define skill-related columns
list_columns = ['analyst_tools', 'libraries', 'cloud', 'databases', 'other', 
                'programming', 'webframeworks', 'os', 'async', 'sync']


# Convert string representation of lists to actual lists
def convert_to_list(value):
    if isinstance(value, str):  # If stored as string
        try:
            result = ast.literal_eval(value)  # Convert string to list
            return result if isinstance(result, list) else []  # Ensure list format
        except (ValueError, SyntaxError):
            return []  # Return empty list if conversion fails
    return value if isinstance(value, list) else []  # Keep existing lists

# Apply conversion only to existing columns
for col in list_columns:
    if col in df.columns:
        df[col] = df[col].apply(convert_to_list)

# Create skill-to-category mapping
skill_to_category = defaultdict(list)

for category in list_columns:
    if category in df.columns:
        for skills in df[category].dropna():
            if isinstance(skills, list) and skills:  # Check if valid list
                for skill in skills:
                    if category not in skill_to_category[skill]:  
                        skill_to_category[skill].append(category)

# Convert mapping to DataFrame
skill_mapping_df = pd.DataFrame(list(skill_to_category.items()), columns=['Skill', 'Categories'])

# Display results
print("\n✅ Skill-to-Category Mapping DataFrame:")
print(skill_mapping_df.head())

# Save the mapping to CSV
skill_mapping_df.to_csv("skill_mapping.csv", index=False)


✅ Actual Column Names: ['job_title_short', 'job_location', 'job_schedule_type', 'job_work_from_home', 'job_country', 'salary_year_avg', 'company_name', 'S.No', 'analyst_tools', 'libraries', 'cloud', 'databases', 'other', 'programming', 'webframeworks', 'os', 'async', 'sync']

✅ Skill-to-Category Mapping DataFrame:
        Skill       Categories
0       excel  [analyst_tools]
1     tableau  [analyst_tools]
2     alteryx  [analyst_tools]
3  powerpoint  [analyst_tools]
4    power bi  [analyst_tools]


LABEL ENCODDING

In [7]:
import pandas as pd
import ast
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder

# Load dataset
df = pd.read_csv("data_clean.csv", encoding="latin1")
df.columns = df.columns.str.replace("'", "").str.strip()

# Ensure column names are clean (remove spaces)
df.columns = df.columns.str.strip()

# Define skill-related columns
list_columns = ['analyst_tools', 'libraries', 'cloud', 'databases', 'other', 
                'programming', 'webframeworks', 'os', 'async', 'sync']

# ✅ 1️⃣ Function to clean skill columns properly
def clean_list_column(value):
    if isinstance(value, str):
        try:
            cleaned_value = ast.literal_eval(value)  # Convert string to list safely
            if isinstance(cleaned_value, list):
                return [item.strip() for item in cleaned_value if isinstance(item, str) and len(item) > 0]  
            else:
                return []
        except (SyntaxError, ValueError):
            return []  # Return empty list if conversion fails
    return value if isinstance(value, list) else []

# Apply cleaning function
for col in list_columns:
    if col in df.columns:
        df[col] = df[col].apply(clean_list_column)

# ✅ 2️⃣ Combine all skills into one column
df['combined_skills'] = df[list_columns].apply(lambda x: sum(x, []), axis=1)

# ✅ 3️⃣ Load & Clean Skill-to-Category Mapping
skill_mapping_df = pd.read_csv("skill_mapping.csv")  

# Ensure 'Skill' and 'Categories' are correctly formatted
skill_mapping_df.dropna(subset=['Skill', 'Categories'], inplace=True)
skill_mapping_df['Skill'] = skill_mapping_df['Skill'].str.strip().str.lower()  
skill_mapping_df['Categories'] = skill_mapping_df['Categories'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

# Convert skill mapping to a dictionary
skill_to_category = {skill: categories for skill, categories in zip(skill_mapping_df['Skill'], skill_mapping_df['Categories'])}

# ✅ 4️⃣ Function to Map Skills to Categories
def map_skills_to_categories(skill_list):
    mapped_categories = set()
    valid_skills = []
    
    for skill in skill_list:
        skill = skill.strip().lower()  # Standardize skill names
        if skill in skill_to_category:  # Only keep valid skills
            mapped_categories.update(skill_to_category[skill])  
            valid_skills.append(skill)  # Store valid skill

    return list(mapped_categories) if mapped_categories else ["Unknown"], valid_skills

# Apply skill category mapping
df[['mapped_skill_categories', 'valid_skills']] = df['combined_skills'].apply(map_skills_to_categories).apply(pd.Series)

# ✅ 5️⃣ Convert Skills into Boolean Features
all_skills = set(skill_mapping_df['Skill'])  # Ensure all mapped skills are included
all_skills.update(["c", "r"])  # Explicitly add 'c' and 'r' if missing

mlb_skills = MultiLabelBinarizer(classes=list(all_skills))  # Set explicit classes
df_skills = pd.DataFrame(mlb_skills.fit_transform(df['valid_skills']), columns=mlb_skills.classes_)

mlb_categories = MultiLabelBinarizer()
df_categories = pd.DataFrame(mlb_categories.fit_transform(df['mapped_skill_categories']), columns=mlb_categories.classes_)

# Merge binary features into the dataset
df = pd.concat([df.drop(columns=list_columns + ['combined_skills', 'mapped_skill_categories', 'valid_skills']), df_skills, df_categories], axis=1)

# ✅ 7️⃣ Apply Label Encoding to Categorical Columns
categorical_columns = ["job_title_short", "job_country", "company_name","job_schedule_type","job_work_from_home"]

label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))  # Convert to string to avoid NaN issues
    label_encoders[col] = dict(zip(le.classes_, le.transform(le.classes_)))

# ✅ 8️⃣ Save Encoding Mappings
with open("encoding_mappings.txt", "w", encoding="utf-8") as f:
    for col, mapping in label_encoders.items():
        f.write(f"Column: {col}\n")
        for key, value in mapping.items():
            f.write(f"  '{key}' -> {value}\n")
        f.write("\n")

# ✅ 9️⃣ Save Processed Dataset
df.to_csv("processed_data.csv", index=False)
print("✅ Dataset processing completed successfully!")


✅ Dataset processing completed successfully!


In [8]:
import re
import json

# Load and parse the encoding mappings
encoding_mappings = {}

with open("encoding_mappings.txt", "r") as f:
    lines = f.readlines()

current_column = None

for line in lines:
    line = line.strip()
    
    # Detect a new column
    if line.startswith("Column:"):
        current_column = line.split(":")[1].strip()
        encoding_mappings[current_column] = {}
    
    # Detect key-value pairs using regex
    elif "->" in line:
        match = re.match(r"'(.+?)'\s*->\s*(\d+)", line)
        if match:
            key, value = match.groups()
            encoding_mappings[current_column][key] = int(value)

# Save as a proper JSON file
with open("encoding_mappings.json", "w") as json_file:
    json.dump(encoding_mappings, json_file, indent=4)

print("✅ Encoding mappings saved as JSON!")


✅ Encoding mappings saved as JSON!


In [18]:
import pandas as pd

# ✅ Load your dataset
data = pd.read_csv("processed_data.csv")

# ✅ List of 11 columns to drop
columns_to_drop = ['analyst_tools', 'async', 'cloud', 'databases', 'libraries', 'os','other', 'programming', 'sync', 'webframeworks']  # Replace with actual column names

# ✅ Drop the columns safely
data = data.drop(columns=columns_to_drop, errors='ignore')

# ✅ Save the cleaned dataset (optional)
data.to_csv("cleaned_data.csv", index=False)

# ✅ Verify the changes
print("Updated Columns:", data.columns)


Updated Columns: Index(['job_title_short', 'job_location', 'job_schedule_type',
       'job_work_from_home', 'job_country', 'salary_year_avg', 'company_name',
       'S.No', 'fortran', 'gatsby',
       ...
       'airflow', 'sql server', 'flow', 'webex', 'tensorflow', 'clojure',
       'couchdb', 'oracle', 'matlab', 'alteryx'],
      dtype='object', length=226)


In [2]:
import pandas as pd

# Load the dataset
df = pd.read_csv("processed_data1.csv")

# List of columns to check for '1' counts
columns_to_check = [
    "excel", "tableau", "alteryx", "powerpoint", "power bi", "word", "visio", "sharepoint",
    "sap", "microstrategy", "sas", "sheets", "looker", "spreadsheet", "dax", "qlik",
    "outlook", "cognos", "ssrs", "datarobot", "ssis", "spss", "splunk", "ms access",
    "gdpr", "spark", "hadoop", "electron", "kafka", "pyspark", "pandas", "airflow",
    "selenium", "numpy", "scikit-learn", "matplotlib", "plotly", "ggplot2", "opencv",
    "tensorflow", "pytorch", "keras", "mxnet", "hugging face", "jupyter", "spring",
    "graphql", "react", "tidyverse", "xamarin", "nltk", "seaborn", "theano", "dplyr",
    "rshiny", "flutter", "mlr", "qt", "cordova", "chainer", "oracle", "aws", "azure",
    "gcp", "snowflake", "databricks", "redshift", "bigquery", "aurora", "vmware",
    "watson", "heroku", "ibm cloud", "firebase", "colocation", "openstack",
    "digitalocean", "mongodb", "mysql", "mariadb", "redis", "dynamodb", "postgresql",
    "sql server", "neo4j", "cassandra", "db2", "elasticsearch", "sqlite", "couchbase",
    "firestore", "couchdb", "kubernetes", "terraform", "git", "jenkins", "gitlab",
    "github", "ansible", "puppet", "docker", "flow", "yarn", "unity", "terminal",
    "atlassian", "chef", "svn", "bitbucket", "pulumi", "codecommit", "unreal",
    "npm", "homebrew", "python", "r", "sql", "t-sql", "golang", "scala", "java",
    "c", "nosql", "go", "c++", "shell", "c#", "javascript", "kotlin", "powershell",
    "php", "mongo", "perl", "bash", "groovy", "vba", "no-sql", "julia", "matlab",
    "html", "assembly", "vb.net", "css", "typescript", "crystal", "solidity",
    "visual basic", "sass", "ruby", "erlang", "swift", "rust", "fortran", "dart",
    "elixir", "haskell", "apl", "delphi", "cobol", "clojure", "lisp", "objective-c",
    "pascal", "lua", "express", "django", "node", "flask", "fastapi", "node.js",
    "phoenix", "react.js", "angular", "blazor", "jquery", "ruby on rails", "laravel",
    "asp.net", "next.js", "vue.js", "svelte", "drupal", "vue", "gatsby", "fastify",
    "asp.net core", "angular.js", "linux", "unix", "centos", "windows", "redhat",
    "ubuntu", "arch", "macos", "suse", "debian", "notion", "jira", "confluence",
    "smartsheet", "asana", "monday.com", "airtable", "workfront", "planner", "trello",
    "wrike", "clickup", "zoom", "slack", "symphony", "unify", "microsoft teams",
    "wire", "twilio", "ringcentral", "webex"
]

# Count the number of 1s in each column
ones_count = df[columns_to_check].sum()

# Filter out columns with less than 50 occurrences of '1'
filtered_columns = ones_count[ones_count >= 50].index.tolist()
print(filtered_columns)
# Required columns to retain
required_columns = ["job_title_short", "job_location", "job_schedule_type", "job_work_from_home",
                    "job_country", "salary_year_avg", "company_name", "S.No"]

# Create a new DataFrame with filtered and required columns
filtered_df = df[filtered_columns + required_columns]

# Save the updated DataFrame to a new CSV file
filtered_df.to_csv("processed_data2.csv", index=False)

print("Updated CSV file 'processed_data2.csv' has been generated successfully.")


['excel', 'tableau', 'alteryx', 'powerpoint', 'power bi', 'word', 'visio', 'sharepoint', 'sap', 'microstrategy', 'sas', 'sheets', 'looker', 'spreadsheet', 'dax', 'qlik', 'outlook', 'cognos', 'ssrs', 'ssis', 'spss', 'splunk', 'ms access', 'gdpr', 'spark', 'hadoop', 'kafka', 'pyspark', 'pandas', 'airflow', 'numpy', 'scikit-learn', 'matplotlib', 'plotly', 'ggplot2', 'opencv', 'tensorflow', 'pytorch', 'keras', 'jupyter', 'spring', 'graphql', 'react', 'nltk', 'seaborn', 'oracle', 'aws', 'azure', 'gcp', 'snowflake', 'databricks', 'redshift', 'bigquery', 'aurora', 'ibm cloud', 'mongodb', 'mysql', 'redis', 'dynamodb', 'postgresql', 'sql server', 'neo4j', 'cassandra', 'db2', 'elasticsearch', 'kubernetes', 'terraform', 'git', 'jenkins', 'gitlab', 'github', 'ansible', 'docker', 'flow', 'terminal', 'atlassian', 'bitbucket', 'python', 'r', 'sql', 't-sql', 'golang', 'scala', 'java', 'c', 'nosql', 'go', 'c++', 'shell', 'c#', 'javascript', 'powershell', 'php', 'mongo', 'perl', 'bash', 'vba', 'no-sql',

In [ ]:
'job_title_short', 'job_location', 'job_schedule_type',
'job_work_from_home', 'job_country', 'salary_year_avg', 'company_name',
'S.No','excel', 'tableau', 'alteryx', 'powerpoint', 'power bi', 'word', 'visio', 'sharepoint',
'sap', 'microstrategy', 'sas', 'sheets', 'looker', 'spreadsheet', 'dax', 'qlik', 'outlook', 'cognos', 'ssrs', 'ssis', 'spss', 'splunk', 
'ms access', 'gdpr', 'spark', 'hadoop', 'kafka', 'pyspark', 'pandas', 'airflow', 'numpy', 'scikit-learn', 'matplotlib', 'plotly', 'ggplot2', 'opencv',
'tensorflow', 'pytorch', 'keras', 'jupyter', 'spring', 'graphql', 'react', 'nltk', 'seaborn', 'oracle', 'aws', 'azure', 'gcp', 'snowflake', 'databricks',
'redshift', 'bigquery', 'aurora', 'ibm cloud', 'mongodb', 'mysql', 'redis', 'dynamodb', 'postgresql', 'sql server', 'neo4j', 'cassandra', 
'db2', 'elasticsearch', 'kubernetes', 'terraform', 'git', 'jenkins', 'gitlab', 'github'
, 'ansible', 'docker', 'flow', 'terminal', 'atlassian', 'bitbucket', 'python', 'r', 'sql', 't-sql', 'golang', 'scala', 'java', 'c', 'nosql', 'go',
'c++', 'shell', 'c#', 'javascript', 'powershell', 'php', 'mongo', 'perl', 'bash', 'vba', 'no-sql', 'julia', 'matlab', 'html', 'assembly',
'css', 'typescript', 'crystal', 'visual basic', 'ruby', 'rust', 'express', 'django', 'flask', 'node.js', 'phoenix', 'angular', 'linux', 
'unix', 'windows', 'jira', 'confluence', 'zoom', 'slack', 'unify'